In [2]:
import pandas as pd
import numpy as np

In [4]:
MAP = '../experiments/exp13/run1/mhc/mapping/split/map/A1072B316' #snakemake.input.mpp #
UMI = '../experiments/exp13/run1/mhc/mapping/split/umi/A1072B316.tsv' #snakemake.input.umi #
#OUTPUT = snakemake.output[0]

# Load
map_df = pd.read_csv(MAP, sep='\t', header=None, names=['seq','hits','barcode','read_id','label'], usecols=[0,1,5,6,8])
umi_df = pd.read_csv(UMI, sep='\t', header=None, names=['barcode','read_ext','umi'], skiprows=1)

map_df['read_id'], map_df['gem'] = map_df.read_id.str.split(r'\s+BX:Z:', expand=True) #.str
map_df['read_ext'] = map_df.read_id.str.rsplit(':', n=1, expand=True)[1]
umi_df['read_ext'] = umi_df.read_ext.astype(str)
map_df['barcode'] = map_df.barcode.str.split(' ', expand=True)[0]

# To make sure that the "best" read is kept one could sort on hits, so we always keep the read with lowest hits.
map_df.drop_duplicates(subset=['read_id','gem'], keep='first', inplace=True)
map_df.drop_duplicates(subset=['read_id'], keep=False, inplace=True)

# To make sure that the "best" read is kept one could sort on umi length, so we always keep the read with full umi seq.
umi_df.replace('', np.nan, inplace=True)
umi_df.dropna(inplace=True)
umi_df.drop_duplicates(subset=['read_ext','barcode'], keep='first', inplace=True)
umi_df.drop_duplicates(subset=['read_ext'], keep=False, inplace=True)

df = pd.merge(map_df, umi_df, on=['read_ext','barcode'])

assert df.duplicated().sum() == 0

df.groupby(['gem','barcode']).umi.size().to_frame()

/Users/herpov/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


,,umi
gem,barcode,
AAACCTGTCCGCATAA-1,A1072B316,1
AAACGGGCACAGATTC-1,A1072B316,1130
AAAGATGGTACCGGCT-1,A1072B316,1
AAAGCAAAGTGTTTGC-1,A1072B316,1120
AAAGTAGGTTCACCTC-1,A1072B316,2
...,...,...
TTGGAACTCATGTAGC-1,A1072B316,1
TTGTAGGAGATTACCC-1,A1072B316,1
TTTACTGGTGATAAGT-1,A1072B316,3518


In [10]:
umi_df

,barcode,read_ext,umi
0,A1072B316,7545,GTCGGGTAAAGA
2,A1072B316,7780,GTCGGGTAAAGA
4,A1072B316,1031,GTCGGGTAAAGA
6,A1072B316,34460,GTCGGGTAAAGA
9,A1072B316,24111,GTCGGGTAAAGA
...,...,...,...
28943,A1072B316,24659,TATTCAGAACAG
29654,A1072B316,25629,GATCCCAGAATA
32481,A1072B316,25864,ACTTTGCGAATG
37478,A1072B316,11209,TCCGTGCCGAGC


In [9]:
df.groupby(['gem','barcode','umi']).size().to_frame().head(30)

0
gem                barcode   umi              
AAACCTGTCCGCATAA-1 A1072B316 GTCGGGTAAAGA    1
AAACGGGCACAGATTC-1 A1072B316 ACATACAAATTA   59
                             ACGGCCCTAAGC   31
                             ACTAGGGGGACA    1
                             ACTCTGGAGGGG    1
                             ACTGTATGAAGA   24
                             ACTGTTTGAAGA    1
                             ATCATTTACATA    5
                             CAAGGCGTTGGG   24
                             CATTTGAAAACA    6
                             CCCATGGGAACA    3
                             CGTAACATCATG  265
                             CGTATCATCATG    2
                             CTACCT          4
                             GTCAAAATAATG   54
                             GTCGGG          1
                             GTCGGGTAAAGA  452
                             GTCGGGTAAAGG    2
                             GTCGGGTAACGA    2
                             TAGTGTCCCAAC  109
                             TCCGGCATCTAA   48
                             TCCGGCCTCTAT    2
                             TCCGTGCCGAGC    1
                             TTAATACATGCT   13
                             TTAGTGTCTATC    4
                             TTCGTTTACGTA   12
                             TTTAGG          4
AAAGATGGTACCGGCT-1 A1072B316 GTCAAAATAATG    1
AAAGCAAAGTGTTTGC-1 A1072B316 AAATACAAATTA    1
                             ACATACAAATTA   68

In [12]:
cnt = df.groupby(['gem','barcode']).umi.nunique().to_frame()

In [13]:
cnt

,,umi
gem,barcode,
AAACCTGTCCGCATAA-1,A1072B316,1
AAACGGGCACAGATTC-1,A1072B316,26
AAAGATGGTACCGGCT-1,A1072B316,1
AAAGCAAAGTGTTTGC-1,A1072B316,30
AAAGTAGGTTCACCTC-1,A1072B316,2
...,...,...
TTGGAACTCATGTAGC-1,A1072B316,1
TTGTAGGAGATTACCC-1,A1072B316,1
TTTACTGGTGATAAGT-1,A1072B316,37


In [18]:
cnt['label'] = df.groupby(['gem','barcode']).label.unique().apply(lambda x: x[0] if len(x)==1 else np.nan)

In [20]:
cnt.reset_index(inplace=True)

In [21]:
cnt

,gem,barcode,umi,label
0,AAACCTGTCCGCATAA-1,A1072B316,1,1
1,AAACGGGCACAGATTC-1,A1072B316,26,1
2,AAAGATGGTACCGGCT-1,A1072B316,1,1
3,AAAGCAAAGTGTTTGC-1,A1072B316,30,1
4,AAAGTAGGTTCACCTC-1,A1072B316,2,1
...,...,...,...,...
325,TTGGAACTCATGTAGC-1,A1072B316,1,1
326,TTGTAGGAGATTACCC-1,A1072B316,1,1
327,TTTACTGGTGATAAGT-1,A1072B316,37,1
328,TTTATGCTCTTGAGGT-1,A1072B316,28,1


In [22]:
cnt.set_index(['gem','barcode'],inplace=True)

In [24]:
cnt.label

gem                 barcode  
AAACCTGTCCGCATAA-1  A1072B316    1
AAACGGGCACAGATTC-1  A1072B316    1
AAAGATGGTACCGGCT-1  A1072B316    1
AAAGCAAAGTGTTTGC-1  A1072B316    1
AAAGTAGGTTCACCTC-1  A1072B316    1
                                ..
TTGGAACTCATGTAGC-1  A1072B316    1
TTGTAGGAGATTACCC-1  A1072B316    1
TTTACTGGTGATAAGT-1  A1072B316    1
TTTATGCTCTTGAGGT-1  A1072B316    1
TTTCCTCCATGTTCCC-1  A1072B316    1
Name: label, Length: 330, dtype: int64

In [2]:
INPUT = '/Users/herpov/Documents/mnt/tuba_home/tcr-pmhc-sc-project/data/exp3_MHC_IONTORRENT/barcode_library/umi.tsv'

In [9]:
df = pd.read_csv(INPUT, sep='\t')

In [18]:
df

,read,gem,ab,sample,SAMPLE,A_PRIM,A_N6,A_OLIGO,ANNEAL,B_OLIGO,B_N6,B_PRIM,sample_id,umi,barcode
1,HBBAF:03366:00950,GCTGCAGGTACCGCTG,A1064B288 <unknown description>,,,AGCCAGCGTC,AGTGGA,CGTAGTAGCTATCTCCTTATGACAA,GTGTGACCTTCCCCAAAAGGCGTAG,CCAAATGGGAGCATGGATTGCCAAG,TCGTTG,GTTACGCCCATGGTACTCCCCATATAAGAAA,,AGTGGATCGTTG,A1064B288
6,HBBAF:02685:00249,GCTGCAGGTACCGCTG,A1064B288 <unknown description>,,,TTCCAGCCAGCGTC,AGTGGA,CGTAGTAGCTATCTCCTTATGACAA,GTGTGACCTTCCCCAAAAGGCGTAG,CCAATGGGAGCATGGATTGCCAAG,TCGTTG,GTTACGCCCATGGTACTCCCCACATAAGAAA,,AGTGGATCGTTG,A1064B288
15,HBBAF:01635:00466,GCTGCAGGTACCGCTG,A1064B288 <unknown description>,,,TTAGCAGCGTC,AGTGGA,CGTAGTAGCTATCTCCTTATGACAA,GTGTGACCTTCCCCAAAAGGCGTAG,CCAAATGGGAGCATGGATTGCCAAG,TCGTTG,GTTACGCCCATGGTACTCCCCATATAAGAAA,,AGTGGATCGTTG,A1064B288
23,HBBAF:00443:01340,GCTGCAGGTACCGCTG,A1064B288 <unknown description>,,,,AGTGGA,CGTAGTAGCTATCTCCTTATGACAA,GTGTGACCTTCCCCAAAAGGCGTAG,CCAATGGGAGCATGGATTGCCAAG,TCGTTG,GTTACGCCCATGGTACTCCCCATATAAGAAA,,AGTGGATCGTTG,A1064B288
24,HBBAF:00382:01540,GCTGCAGGTACCGCTG,A1064B288 <unknown description>,,,TTAGCCAGCGTC,AGTGGA,CGTAGTAGCTATCTCCTTATGACAA,GTGTGACCTTCCCCAAAAGGCGTAG,CCAAATGGGAGCATGGATTGCCAAG,TCGTTG,GTTACGCCCATGGTACTCCCCATATAAGAAA,,AGTGGATCGTTG,A1064B288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245140,HBBAF:00621:02112,CGCGTTTTCTAACGGT,A4000B302 <unknown description>,,AGGCTGAA,GAAGTAAAAGATCCCAGGTTTC,CTAGGC,CTGCGGGTATAGGACCTTGATAACC,GTGTGACCTTCCCCAAAAGGCGTAG,TACAATTCAAGCCTCCCCC,ATATAA,GTAGTACAATTCAAGCCTCCCCCATATAAGAAA,bc25mer_51469,CTAGGCATATAA,A4000B302
245141,HBBAF:00373:00554,CGCGTTTTCTAACGGT,A4000B302 <unknown description>,,AGGCTGAA,GAAGTAAAAGATCCCAGGTTTC,GAGTGT,CTGCGGGTATAGGACCTTGATAACC,GTGTGACCTTCCCCAAAAGGCGTAG,TACAATTCAAGCCTCCCCC,ATATAA,GTAGTACAATTCAAGCCTCCCCCATATAAGAAA,bc25mer_51469,GAGTGTATATAA,A4000B302
245143,HBBAF:02655:02122,ACTTTCAAGATCCGAG,A4000B302 <unknown description>,,TCAAGC,GGTCAGCCAC,GTCCTT,CTGCGGGTATAGGACCTTGATAAC,GTGTGACCTTCCCCAAAAGGCGTAG,TACAATTCAAGCCTCCCGGTCAGCC,ACTATA,GTTACGCCCATGGTACTCGCCCATATAAGAAA,bc25mer_23398,GTCCTTACTATA,A4000B302
245144,HBBAF:01027:02674,ACTTTCAAGATCCGAG,A4000B302 <unknown description>,,TCAAGC,GGTCAGCCAC,GTCCTT,CTGCGGGTATAGGACCTTGATAAC,GTGTGACCTTCCCCAAAAGGCGTAG,TACAATTCAAGCCTCCCGGTCAGCC,ACTATA,GTTACGCCCATGGTACTCGCCCATATAAGAAA,bc25mer_23398,GTCCTTACTATA,A4000B302


In [17]:
df['barcode'] = df.ab.str.split(' ', expand=True)[0]

In [10]:
df.fillna('', inplace=True)
#df = 
df = df[(df.A_N6.apply(lambda x: len(str(x))==6)) & (df.B_N6.apply(lambda x: len(str(x))==6))].copy()
df['umi'] = df.A_N6 + df.B_N6

In [25]:
c = df.groupby(['gem','umi']).size().to_frame().rename(columns={0:'freq'})
c

freq
gem              umi               
AAACCTGAGGAACTGC AAAAACCGCCTA     4
                 GCCCGAAGGGGC    68
                 GCCCGAAGGGGG     1
                 GCCCGGCGCCTA     2
                 GGATGACGCCTA     1
...                             ...
TTTGTCATCAACGGCC TACAACTACTCG     1
                 TCAGCCCGGCGT     1
TTTGTCATCAAGCCTA AGCGGTGGAGAC     2
                 CGTGTGGGAGAC     1
                 GCGGTAGGAGAC    27

[25218 rows x 1 columns]

In [37]:
c = df.groupby(['gem','umi']).barcode.unique().apply(lambda x: x[0] if len(x)==1 else np.nan).to_frame() #.isna().any()
c

barcode
gem              umi                    
AAACCTGAGGAACTGC AAAAACCGCCTA  A1067B297
                 GCCCGAAGGGGC  A1067B297
                 GCCCGAAGGGGG  A1067B297
                 GCCCGGCGCCTA  A4000B297
                 GGATGACGCCTA  A4000B297
...                                  ...
TTTGTCATCAACGGCC TACAACTACTCG  A4000B288
                 TCAGCCCGGCGT  A1071B288
TTTGTCATCAAGCCTA AGCGGTGGAGAC  A1064B294
                 CGTGTGGGAGAC  A1064B294
                 GCGGTAGGAGAC  A1064B294

[25218 rows x 1 columns]

In [40]:
dct = c.reset_index().groupby(['gem','barcode']).umi.count()
c['freq'] = c.reset_index().set_index(['gem','barcode']).index.map(dct)

In [44]:
m = (c
     .reset_index()
     .drop_duplicates(subset=['gem','barcode','freq'])
     .pivot(index='barcode',columns='gem',values='freq'))

In [45]:
m

gem,AAACCTGAGGAACTGC,AAACCTGAGGTCATCT,AAACCTGAGTCATCCA,AAACCTGAGTTCGATC,AAACCTGCAAGCCATT,AAACCTGCAATGCCAT,AAACCTGCAGCATGAG,AAACCTGCAGTAACGG,AAACCTGCATCTCCCA,AAACCTGGTCTTGTCC,...,TTTGTCAAGATCCCGC,TTTGTCAAGGATGGAA,TTTGTCAAGTCGCCGT,TTTGTCACAACACGCC,TTTGTCACACGACGAA,TTTGTCACAGCAGTTT,TTTGTCAGTCTAGAGG,TTTGTCAGTGAGGCTA,TTTGTCATCAACGGCC,TTTGTCATCAAGCCTA
barcode,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1064B288,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN
A1064B289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1064B290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1064B291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A4000B298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A4000B299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A4000B300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# save columns
# save rows
# save matrix